# Install Detectron2

**Make sure you have pytorch with CUDA support**

In [ ]:
# install pyyaml==5.1 to prevent dependency issue
!python -m pip install pyyaml==5.1
!pip install -U gdown
# install detectron2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

**Check detectron2 installation**

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

# Load Trained Model and Perform Inference

In [3]:
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
!gdown https://drive.google.com/uc?id=1tYTusIvB0QQn2ZlbcZLeTdym4gB5FD5k
!gdown https://drive.google.com/uc?id=1n00pi5KItkbqgFDsY2gOVAjGpCphpqBi
!gdown https://drive.google.com/uc?id=1SiuRUpvjUbwV4G_9YzBfxPuVOMW_Q0y-
!gdown https://drive.google.com/uc?id=1yThjIq-VChU-rH2L3mZkbmJsEE1fvIGA

!unzip data.zip

In [22]:
im = cv2.imread('/content/classify.jpg')

In [19]:
cfg = get_cfg()
cfg.merge_from_file("/content/apples_TRAIN_3_fpn50.yaml")
cfg.MODEL.WEIGHTS = "/content/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

In [23]:
outputs = predictor(im)

# Train Random Forest Classifier to Classify Apples' Color

In [ ]:
from sklearn.ensemble import RandomForestClassifier

image_dir = "/content/data"
labels = []
# Get the list of all files in the directory
all_files = os.listdir(image_dir)

image_paths = [os.path.join(image_dir, file_name) for file_name in all_files if file_name.endswith('.jpg')]
labels = [file_name.split('_')[0] for file_name in all_files if file_name.endswith('.jpg')]
# Load images and calculate color histograms
histograms = []
for image_path in image_paths:
    image = cv2.imread(image_path)

    histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    histogram = cv2.normalize(histogram, histogram).flatten()

    histograms.append(histogram)

X = np.array(histograms)
y = np.array(labels)

rf_model = RandomForestClassifier()
rf_model.fit(X, y)


**Check all apple instances' color using random forest and save the images**

In [29]:
from sklearn.ensemble import RandomForestClassifier

instances = outputs["instances"].to("cpu")

out_dir = '/content/output'
os.makedirs(out_dir, exist_ok=True)

counter = {
    'red': 0,
    'green': 0,
    'yellow': 0
    }

# Loop over each instance
for i, pred_mask in enumerate(instances.pred_masks):

    mask = pred_mask.numpy().astype(bool)

    segmented_image = im.copy()
    segmented_image[~mask] = 0

    ys, xs = np.where(mask)
    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()

    cropped_image = segmented_image[y_min:y_max, x_min:x_max]

    test_histogram = cv2.calcHist([cropped_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    test_histogram = cv2.normalize(test_histogram, test_histogram).flatten()

    test_histogram = test_histogram.reshape(1, -1)
    color = rf_model.predict(test_histogram)[0]
    counter[color] += 1

    cv2.imwrite(os.path.join(out_dir, f"{color}_{counter[color]}.jpg"), cropped_image)